<a href="https://colab.research.google.com/github/ThienNguyen3001/Graph-Theory-ArangoDB-Final/blob/main/Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Chuẩn bị

* Gọi môi trường

In [ ]:
%%capture
!git clone https://github.com/ThienNguyen3001/Graph-Theory-ArangoDB-Final.git
!rsync -av Graph-Theory-ArangoDB-Final/ ./ --exclude=.git
!pip3 install pyarango
!pip3 install "python-arango>=5.0"

In [2]:
import json
import requests
import sys
import time

import oasis
from pyArango.connection import *
from arango import ArangoClient

* **Tạo cơ sở dữ liệu tạm thời**


In [3]:
# Retrieve tmp credentials from ArangoDB Tutorial Service
login = oasis.getTempCredentials(tutorialName='test',
                                 credentialProvider='https://tutorials.arangodb.cloud:8529/_db/_system/tutorialDB/tutorialDB')

# Connect to the temp database
conn = oasis.connect(login)
db = conn[login['dbName']]

Sử dụng lại thông tin đăng nhập được lưu trong bộ nhớ đệm.


* **Xem thông tin DB đã tạo**


In [4]:
print('https://{}:{}'.format(login['hostname'], login['port']))
print('Username: ' + login['username'])
print('Password: ' + login['password'])
print('Database: ' + login['dbName'])


https://tutorials.arangodb.cloud:8529
Username: TUTl7g60voxqjohkvrpsdvonm
Password: TUTtm65hw88wckset5ckp4gq
Database: TUT7dbsfgdpd4tz6qzv7o8ui


In [5]:
print(type(login), login, sep='\n')

<class 'dict'>
{'dbName': 'TUT7dbsfgdpd4tz6qzv7o8ui', 'username': 'TUTl7g60voxqjohkvrpsdvonm', 'password': 'TUTtm65hw88wckset5ckp4gq', 'hostname': 'tutorials.arangodb.cloud', 'port': 8529}


# 2. Tạo dữ liệu AQL

* **Tạo collection "Collection" trong DB**

Các bộ sưu tập trong ArangoDB. Chúng chỉ được thao tác thông qua các ngôn ngữ Ngôn ngữ truy vấn Arango NoSQL (AQL). Các hướng dẫn bên dưới tạo ra một bộ sưu tập và có thể liên quan đến các hướng dẫn DDL của ngôn ngữ SQL. Trong ví dụ này, nhóm sẽ tạo các nhân vật trong loạt phim "*Game of Thrones*".

In [ ]:
collecName = 'Characters'
if collecName not in db.collections:
	db.createCollection(name=collecName)
else:
	print(f"Collection '{collecName}' đã tồn tại")

Collection 'Characters' already exists.


* **Thêm data bằng Insert**

In [7]:
insert_query = """
INSERT {
    "name": "Ned",
    "surname": "Stark",
    "alive": true,
    "age": 41,
    "traits": ["A","H","C","N","P"]
} INTO Characters
"""

db.AQLQuery(insert_query)

* Lấy tất cả các nhân vật trong bản ghi

In [8]:
all_characters = """
FOR c IN Characters
    RETURN c
"""

query_result = db.AQLQuery(all_characters, rawResults=True)

for doc in  query_result:
    print(doc)
    print()

{'_key': '266280094263', '_id': 'Characters/266280094263', '_rev': '_i4m-j2u---', 'name': 'Ned', 'surname': 'Stark', 'alive': True, 'age': 41, 'traits': ['A', 'H', 'C', 'N', 'P']}

{'_key': '266280094265', '_id': 'Characters/266280094265', '_rev': '_i4m-kci--A', 'name': 'Robert', 'surname': 'Baratheon', 'alive': False, 'traits': ['A', 'H', 'C']}

{'_key': '266280094266', '_id': 'Characters/266280094266', '_rev': '_i4m-kci--B', 'name': 'Jaime', 'surname': 'Lannister', 'alive': True, 'age': 36, 'traits': ['A', 'F', 'B']}

{'_key': '266280094267', '_id': 'Characters/266280094267', '_rev': '_i4m-kci--C', 'name': 'Catelyn', 'surname': 'Stark', 'alive': False, 'age': 40, 'traits': ['D', 'H', 'C']}

{'_key': '266280094268', '_id': 'Characters/266280094268', '_rev': '_i4m-kci--D', 'name': 'Cersei', 'surname': 'Lannister', 'alive': True, 'age': 36, 'traits': ['H', 'E', 'F']}

{'_key': '266280094269', '_id': 'Characters/266280094269', '_rev': '_i4m-kci--E', 'name': 'Daenerys', 'surname': 'Targar

* **Chèn các bản ghi**

In [9]:
insert_query = """
LET data = [
    { "name": "Robert", "surname": "Baratheon", "alive": false, "traits": ["A","H","C"] },
    { "name": "Jaime", "surname": "Lannister", "alive": true, "age": 36, "traits": ["A","F","B"] },
    { "name": "Catelyn", "surname": "Stark", "alive": false, "age": 40, "traits": ["D","H","C"] },
    { "name": "Cersei", "surname": "Lannister", "alive": true, "age": 36, "traits": ["H","E","F"] },
    { "name": "Daenerys", "surname": "Targaryen", "alive": true, "age": 16, "traits": ["D","H","C"] },
    { "name": "Jorah", "surname": "Mormont", "alive": false, "traits": ["A","B","C","F"] },
    { "name": "Petyr", "surname": "Baelish", "alive": false, "traits": ["E","G","F"] },
    { "name": "Viserys", "surname": "Targaryen", "alive": false, "traits": ["O","L","N"] },
    { "name": "Jon", "surname": "Snow", "alive": true, "age": 16, "traits": ["A","B","C","F"] },
    { "name": "Sansa", "surname": "Stark", "alive": true, "age": 13, "traits": ["D","I","J"] },
    { "name": "Arya", "surname": "Stark", "alive": true, "age": 11, "traits": ["C","K","L"] },
    { "name": "Robb", "surname": "Stark", "alive": false, "traits": ["A","B","C","K"] },
    { "name": "Theon", "surname": "Greyjoy", "alive": true, "age": 16, "traits": ["E","R","K"] },
    { "name": "Bran", "surname": "Stark", "alive": true, "age": 10, "traits": ["L","J"] },
    { "name": "Joffrey", "surname": "Baratheon", "alive": false, "age": 19, "traits": ["I","L","O"] },
    { "name": "Sandor", "surname": "Clegane", "alive": true, "traits": ["A","P","K","F"] },
    { "name": "Tyrion", "surname": "Lannister", "alive": true, "age": 32, "traits": ["F","K","M","N"] },
    { "name": "Khal", "surname": "Drogo", "alive": false, "traits": ["A","C","O","P"] },
    { "name": "Tywin", "surname": "Lannister", "alive": false, "traits": ["O","M","H","F"] },
    { "name": "Davos", "surname": "Seaworth", "alive": true, "age": 49, "traits": ["C","K","P","F"] },
    { "name": "Samwell", "surname": "Tarly", "alive": true, "age": 17, "traits": ["C","L","I"] },
    { "name": "Stannis", "surname": "Baratheon", "alive": false, "traits": ["H","O","P","M"] },
    { "name": "Melisandre", "alive": true, "traits": ["G","E","H"] },
    { "name": "Margaery", "surname": "Tyrell", "alive": false, "traits": ["M","D","B"] },
    { "name": "Jeor", "surname": "Mormont", "alive": false, "traits": ["C","H","M","P"] },
    { "name": "Bronn", "alive": true, "traits": ["K","E","C"] },
    { "name": "Varys", "alive": true, "traits": ["M","F","N","E"] },
    { "name": "Shae", "alive": false, "traits": ["M","D","G"] },
    { "name": "Talisa", "surname": "Maegyr", "alive": false, "traits": ["D","C","B"] },
    { "name": "Gendry", "alive": false, "traits": ["K","C","A"] },
    { "name": "Ygritte", "alive": false, "traits": ["A","P","K"] },
    { "name": "Tormund", "surname": "Giantsbane", "alive": true, "traits": ["C","P","A","I"] },
    { "name": "Gilly", "alive": true, "traits": ["L","J"] },
    { "name": "Brienne", "surname": "Tarth", "alive": true, "age": 32, "traits": ["P","C","A","K"] },
    { "name": "Ramsay", "surname": "Bolton", "alive": true, "traits": ["E","O","G","A"] },
    { "name": "Ellaria", "surname": "Sand", "alive": true, "traits": ["P","O","A","E"] },
    { "name": "Daario", "surname": "Naharis", "alive": true, "traits": ["K","P","A"] },
    { "name": "Missandei", "alive": true, "traits": ["D","L","C","M"] },
    { "name": "Tommen", "surname": "Baratheon", "alive": true, "traits": ["I","L","B"] },
    { "name": "Jaqen", "surname": "H'ghar", "alive": true, "traits": ["H","F","K"] },
    { "name": "Roose", "surname": "Bolton", "alive": true, "traits": ["H","E","F","A"] },
    { "name": "The High Sparrow", "alive": true, "traits": ["H","M","F","O"] }
]

FOR d IN data
    INSERT d INTO Characters
"""

db.AQLQuery(insert_query)

* Truy vấn cơ bản

In [10]:
all_characters_names = """
FOR name IN Characters
    RETURN name.name
"""

query_result = db.AQLQuery(all_characters_names, rawResults=True)

# Lấy ra chỉ 10 tên
for i in range(10):
  print(query_result[i])

Ned
Robert
Jaime
Catelyn
Cersei
Daenerys
Jorah
Petyr
Viserys
Jon


In [11]:
list(query_result)

['Ned',
 'Robert',
 'Jaime',
 'Catelyn',
 'Cersei',
 'Daenerys',
 'Jorah',
 'Petyr',
 'Viserys',
 'Jon',
 'Sansa',
 'Arya',
 'Robb',
 'Theon',
 'Bran',
 'Joffrey',
 'Sandor',
 'Tyrion',
 'Khal',
 'Tywin',
 'Davos',
 'Samwell',
 'Stannis',
 'Melisandre',
 'Margaery',
 'Jeor',
 'Bronn',
 'Varys',
 'Shae',
 'Talisa',
 'Gendry',
 'Ygritte',
 'Tormund',
 'Gilly',
 'Brienne',
 'Ramsay',
 'Ellaria',
 'Daario',
 'Missandei',
 'Tommen',
 'Jaqen',
 'Roose',
 'The High Sparrow',
 'Ned',
 'Robert',
 'Jaime',
 'Catelyn',
 'Cersei',
 'Daenerys',
 'Jorah',
 'Petyr',
 'Viserys',
 'Jon',
 'Sansa',
 'Arya',
 'Robb',
 'Theon',
 'Bran',
 'Joffrey',
 'Sandor',
 'Tyrion',
 'Khal',
 'Tywin',
 'Davos',
 'Samwell',
 'Stannis',
 'Melisandre',
 'Margaery',
 'Jeor',
 'Bronn',
 'Varys',
 'Shae',
 'Talisa',
 'Gendry',
 'Ygritte',
 'Tormund',
 'Gilly',
 'Brienne',
 'Ramsay',
 'Ellaria',
 'Daario',
 'Missandei',
 'Tommen',
 'Jaqen',
 'Roose',
 'The High Sparrow',
 'Ned',
 'Robert',
 'Jaime',
 'Catelyn',
 'Cersei',
 '

* **Truy vấn có điều kiện**


In [12]:
find_ned_query = """
FOR c IN Characters
    FILTER c.alive == false
    RETURN c.name
"""
query_result = db.AQLQuery(find_ned_query, rawResults=True)
for doc in  query_result:
    print(doc, sep='\n')

Robert
Catelyn
Jorah
Petyr
Viserys
Robb
Joffrey
Khal
Tywin
Stannis
Margaery
Jeor
Shae
Talisa
Gendry
Ygritte
Robert
Catelyn
Jorah
Petyr
Viserys
Robb
Joffrey
Khal
Tywin
Stannis
Margaery
Jeor
Shae
Talisa
Gendry
Ygritte
Robert
Catelyn
Jorah
Petyr
Viserys
Robb
Joffrey
Khal
Tywin
Stannis
Margaery
Jeor
Shae
Talisa
Gendry
Ygritte


In [13]:
find_ned_query = """
FOR c IN Characters
    FILTER c.alive == true AND c.age >= 18
    RETURN c.name
"""
query_result = db.AQLQuery(find_ned_query, rawResults=True)

for doc in  query_result:
    print(doc, sep='\n')

Ned
Jaime
Cersei
Tyrion
Davos
Brienne
Ned
Jaime
Cersei
Tyrion
Davos
Brienne
Ned
Jaime
Cersei
Tyrion
Davos
Brienne


In [14]:
find_ned_query = """
FOR c IN Characters
    FILTER c.alive == false AND c.age >= 18
    RETURN { Name: c.name, Surname: c.surname, Age: c.age }
"""
query_result = db.AQLQuery(find_ned_query, rawResults=True)

for doc in  query_result:
    print(doc, sep='\n')

{'Name': 'Catelyn', 'Surname': 'Stark', 'Age': 40}
{'Name': 'Joffrey', 'Surname': 'Baratheon', 'Age': 19}
{'Name': 'Catelyn', 'Surname': 'Stark', 'Age': 40}
{'Name': 'Joffrey', 'Surname': 'Baratheon', 'Age': 19}
{'Name': 'Catelyn', 'Surname': 'Stark', 'Age': 40}
{'Name': 'Joffrey', 'Surname': 'Baratheon', 'Age': 19}


In [15]:
find_ned_query = """
FOR c IN Characters
    FILTER c.surname == "Bolton"
    RETURN {Name: c.name, Surname: c.surname}
"""
query_result = db.AQLQuery(find_ned_query, rawResults=True)
for doc in  query_result:
    print(doc, sep='\n')

{'Name': 'Ramsay', 'Surname': 'Bolton'}
{'Name': 'Roose', 'Surname': 'Bolton'}
{'Name': 'Ramsay', 'Surname': 'Bolton'}
{'Name': 'Roose', 'Surname': 'Bolton'}
{'Name': 'Ramsay', 'Surname': 'Bolton'}
{'Name': 'Roose', 'Surname': 'Bolton'}


In [16]:

sample_chars_query = """
FOR c IN Characters
    LIMIT 10
    RETURN c.name
"""

query_result = db.AQLQuery(sample_chars_query, rawResults=True)

for doc in  query_result:
    print(doc, sep='\n')

Ned
Robert
Jaime
Catelyn
Cersei
Daenerys
Jorah
Petyr
Viserys
Jon


In [17]:

sample_chars_query = """
FOR c IN Characters
    SORT c.name
    LIMIT 10
    RETURN { Name: c.name, Surname: c.surname, Age: c.age }
"""

query_result = db.AQLQuery(sample_chars_query, rawResults=True)

for doc in  query_result:
    print(doc, sep='\n')

{'Name': 'Arya', 'Surname': 'Stark', 'Age': 11}
{'Name': 'Arya', 'Surname': 'Stark', 'Age': 11}
{'Name': 'Arya', 'Surname': 'Stark', 'Age': 11}
{'Name': 'Bran', 'Surname': 'Stark', 'Age': 10}
{'Name': 'Bran', 'Surname': 'Stark', 'Age': 10}
{'Name': 'Bran', 'Surname': 'Stark', 'Age': 10}
{'Name': 'Brienne', 'Surname': 'Tarth', 'Age': 32}
{'Name': 'Brienne', 'Surname': 'Tarth', 'Age': 32}
{'Name': 'Brienne', 'Surname': 'Tarth', 'Age': 32}
{'Name': 'Bronn', 'Surname': None, 'Age': None}


# 3. Tạo đồ thị

## Lấy data từ "shortest path data"

In [18]:
from pyArango.collection import Collection, Field, Edges
from pyArango.graph import Graph, EdgeDefinition

# Định nghĩa Collections
class Places(Collection):
    _fields = {
        "label": Field()
    }

class Connect(Edges):
    _fields = {
        "travelTime": Field()
    }

# Định nghĩa Graph
class TravelGraph(Graph):
    _edgeDefinitions = [
        EdgeDefinition("Connect", fromCollections=["Places"], toCollections=["Places"]),
        ]
    _orphanedCollections = []


if "Places" not in db.collections:
    db.createCollection(className="Places")

if "Connect" not in db.collections:
    db.createCollection(className="Connect")

# Tạo graph
if "TravelGraph" not in db.graphs:
    travel_graph = db.createGraph("TravelGraph")
else:
    travel_graph = db.graphs["TravelGraph"]


import json

# Đọc dữ liệu từ tệp JSON
with open("./shortest_path_data/places.json", "r") as f:
    places_data = json.load(f)
    db["Places"].importBulk(places_data)


with open("./shortest_path_data/connect.json", "r") as f:
    connections_data = json.load(f)
    db["Connect"].importBulk(connections_data)

print("Tạo đồ thị thành công")

Tạo đồ thị thành công


In [19]:
def get_graph_data(directed = False):
    places_query = "FOR v IN Places RETURN {key: v._key, label: v.label}"
    connections_query = "FOR e IN Connect RETURN {from: e._from, to: e._to, travelTime: e.travelTime}"

    places = {doc["key"]: doc["label"] for doc in db.AQLQuery(places_query, rawResults=True)}
    connections = db.AQLQuery(connections_query, rawResults=True)

    # Xây dựng danh sách kề với trọng số
    adjacency_list = {key: [] for key in places.keys()}
    for connection in connections:
        from_key = connection["from"].split("/")[-1]
        to_key = connection["to"].split("/")[-1]
        travel_time = connection["travelTime"]
        adjacency_list[from_key].append((to_key, travel_time))
        if not directed:
            adjacency_list[to_key].append((from_key, travel_time))
    return adjacency_list, places

adjacency_list, places = get_graph_data()
for key, value in adjacency_list.items():
    print(f"{key}: {value}")

Inverness: [('Aberdeen', 2.5), ('Edmonton', 4), ('StAndrews', 4.5)]
Aberdeen: [('Inverness', 2.5), ('Leuchars', 1.5)]
Leuchars: [('Aberdeen', 1.5), ('Edinburgh', 3), ('StAndrews', 0.2), ('Carlisle', 1.5)]
StAndrews: [('Leuchars', 0.2), ('Inverness', 4.5)]
Edinburgh: [('Leuchars', 3), ('Glasgow', 1), ('York', 4)]
Glasgow: [('Edinburgh', 1), ('Carlisle', 1)]
York: [('Edinburgh', 4), ('Carlisle', 2.5), ('London', 1.8)]
Carlisle: [('Glasgow', 1), ('York', 2.5), ('Birmingham', 1), ('Leuchars', 1.5), ('Cologne', 1.25)]
Birmingham: [('Carlisle', 1), ('London', 2.5), ('Toronto', 8.25)]
London: [('Birmingham', 2.5), ('York', 1.8), ('Brussels', 2.5)]
Brussels: [('London', 2.5), ('Cologne', 1.5)]
Cologne: [('Brussels', 1.5), ('Carlisle', 1.25)]
Toronto: [('Winnipeg', 36), ('Birmingham', 8.25)]
Winnipeg: [('Toronto', 36), ('Saskatoon', 12), ('Vancouver', 4.75)]
Saskatoon: [('Winnipeg', 12), ('Edmonton', 12), ('Jasper', 6)]
Edmonton: [('Saskatoon', 12), ('Jasper', 6), ('Inverness', 4)]
Jasper: [('E

## Đường đi ngắn nhất

In [20]:
query = """
FOR p IN OUTBOUND K_SHORTEST_PATHS 'Places/Aberdeen' TO 'Places/London'
GRAPH 'TravelGraph'
    LIMIT 1
    RETURN {
        places: p.vertices[*].label,
        travelTimeTotal: SUM(p.edges[*].travelTime)
    }
"""

query_result = db.AQLQuery(query, rawResults=True)

for doc in query_result:
    print(doc, sep='\n')

{'places': ['Aberdeen', 'Leuchars', 'Carlisle', 'York', 'London'], 'travelTimeTotal': 7.3}


## Thuật toán Prim

In [21]:
from GraphAlgo import *

start_vertex = 'Inverness'
mst_edges, total_weight = prim(adjacency_list, start_vertex)

# Hiển thị kết quả
print("Cây khung nhỏ nhất:", mst_edges)
print("Tổng trọng số:", total_weight)

Cây khung nhỏ nhất: [('Inverness', 'Aberdeen', 2.5), ('Aberdeen', 'Leuchars', 1.5), ('Leuchars', 'StAndrews', 0.2), ('Leuchars', 'Carlisle', 1.5), ('Carlisle', 'Birmingham', 1), ('Carlisle', 'Glasgow', 1), ('Glasgow', 'Edinburgh', 1), ('Carlisle', 'Cologne', 1.25), ('Cologne', 'Brussels', 1.5), ('Birmingham', 'London', 2.5), ('London', 'York', 1.8), ('Inverness', 'Edmonton', 4), ('Edmonton', 'Jasper', 6), ('Jasper', 'Saskatoon', 6), ('Birmingham', 'Toronto', 8.25), ('Saskatoon', 'Winnipeg', 12), ('Winnipeg', 'Vancouver', 4.75)]
Tổng trọng số: 56.75


## Kruskal

In [22]:
mst_edges, total_weight = kruskal(adjacency_list)

# Hiển thị kết quả
print("Cây khung nhỏ nhất:", mst_edges)
print("Tổng trọng số:", total_weight)

Cây khung nhỏ nhất: [('Leuchars', 'StAndrews', 0.2), ('Edinburgh', 'Glasgow', 1), ('Glasgow', 'Carlisle', 1), ('Carlisle', 'Birmingham', 1), ('Carlisle', 'Cologne', 1.25), ('Aberdeen', 'Leuchars', 1.5), ('Leuchars', 'Carlisle', 1.5), ('Brussels', 'Cologne', 1.5), ('York', 'London', 1.8), ('Inverness', 'Aberdeen', 2.5), ('York', 'Carlisle', 2.5), ('Inverness', 'Edmonton', 4), ('Winnipeg', 'Vancouver', 4.75), ('Saskatoon', 'Jasper', 6), ('Edmonton', 'Jasper', 6), ('Birmingham', 'Toronto', 8.25), ('Winnipeg', 'Saskatoon', 12)]
Tổng trọng số: 56.75


## Tô màu

## Lấy data từ "components data"

In [23]:
from pyArango.collection import Collection, Field, Edges
from pyArango.graph import Graph, EdgeDefinition

# Định nghĩa Collections
class Components(Collection):
    _fields = {
        "_key": Field()
    }

class Connections(Edges):
    pass

# Định nghĩa Graph
class ComponentsGraph(Graph):
    _edgeDefinitions = [
        EdgeDefinition("Connections", fromCollections=["Components"], toCollections=["Components"]),
        ]
    _orphanedCollections = []


if "Components" not in db.collections:
    db.createCollection(className="Components")

if "Connections" not in db.collections:
    db.createCollection(className="Connections")

# Tạo graph
if "ComponentsGraph" not in db.graphs:
    travel_graph = db.createGraph("ComponentsGraph")
else:
    travel_graph = db.graphs["ComponentsGraph"]


import json

# Đọc dữ liệu từ tệp JSON
with open("./components_data/components.json", "r") as f:
    places_data = json.load(f)
    db["Components"].importBulk(places_data)


with open("./components_data/connections.json", "r") as f:
    connections_data = json.load(f)
    db["Connections"].importBulk(connections_data)

print("Tạo đồ thị thành công")

Tạo đồ thị thành công


In [24]:
def get_graph_data(directed=False):
    places_query = "FOR v IN Components RETURN v._key"
    connections_query = "FOR e IN Connections RETURN {from: e._from, to: e._to}"

    places = {doc for doc in db.AQLQuery(places_query, rawResults=True)}
    connections = db.AQLQuery(connections_query, rawResults=True)

    # Xây dựng danh sách kề
    adjacency_list = {key: [] for key in places}
    for connection in connections:
        from_key = connection["from"].split("/")[-1]
        to_key = connection["to"].split("/")[-1]
        adjacency_list[from_key].append(to_key)
        if not directed:
            adjacency_list[to_key].append(from_key)
    return adjacency_list, places

adjacency_list, places = get_graph_data()
for key, value in adjacency_list.items():
    print(f"{key}: {value}")

B4: ['B2', 'B3', 'B5']
C7: ['C4', 'C5', 'C8']
A2: ['A1', 'A3']
A1: ['A2', 'A4']
B17: ['B15', 'B18', 'B19']
C3: ['C2', 'C4']
C4: ['C3', 'C5', 'C7']
B21: ['B20']
B12: ['B11', 'B13']
C10: ['C8']
B22: ['B20']
B6: ['B3', 'B7']
C5: ['C4', 'C6', 'C7']
B13: ['B12', 'B20', 'B14']
B20: ['B13', 'B21', 'B22']
B8: ['B7']
B14: ['B13', 'B15']
C8: ['C7', 'C9', 'C10']
B3: ['B1', 'B6', 'B4']
C1: ['C2']
B10: ['B7', 'B11']
B16: ['B15']
A3: ['A2', 'A4']
B15: ['B14', 'B16', 'B17']
B7: ['B6', 'B8', 'B9', 'B10', 'B19']
B9: ['B7']
C6: ['C5']
C9: ['C8']
B11: ['B10', 'B12']
A4: ['A3', 'A1']
B5: ['B4']
B19: ['B7', 'B17']
C2: ['C1', 'C3']
B2: ['B4']
B18: ['B17']
B1: ['B3']


In [25]:
start = 'A2'

free_ser        = pd.Series(1, index = adjacency_list.keys())
free_ser[start] = 0

seq = []
dfs(adjacency_list, free_ser, start, seq)
seq = ' --> '.join(seq)
print(f'Chuỗi thứ tự duyệt đồ thị từ đỉnh [{start}]:\n{seq}')

Chuỗi thứ tự duyệt đồ thị từ đỉnh [A2]:
A2 --> A1 --> A4 --> A3


In [26]:
components = dfs_all_components(adjacency_list)
for c in components:
    print(f'Thành phần liên thông #{components.index(c) + 1} = {c}')

Thành phần liên thông #1 = ['B4', 'B2', 'B3', 'B1', 'B6', 'B7', 'B8', 'B9', 'B10', 'B11', 'B12', 'B13', 'B20', 'B21', 'B22', 'B14', 'B15', 'B16', 'B17', 'B18', 'B19', 'B5']
Thành phần liên thông #2 = ['C7', 'C4', 'C3', 'C2', 'C1', 'C5', 'C6', 'C8', 'C9', 'C10']
Thành phần liên thông #3 = ['A2', 'A1', 'A4', 'A3']


In [27]:
# Sử dụng hàm
start = 'C1'
hamilton_cycle = find_hamilton_cycle(adjacency_list, start)

# In kết quả
if hamilton_cycle:
    print(f"Chu trình Hamilton bắt đầu từ đỉnh [{start}]:")
    print(' --> '.join(hamilton_cycle))
    print(f"\nĐộ dài chu trình: {len(hamilton_cycle)} đỉnh")
else:
    print(f"Không tìm thấy chu trình Hamilton bắt đầu từ đỉnh [{start}]")

# Kiểm tra tính hợp lệ của chu trình
if hamilton_cycle:
    is_valid = is_valid_hamilton_cycle(hamilton_cycle, adjacency_list)
    print(f"\nChu trình Hamilton hợp lệ: {is_valid}")

Không tìm thấy chu trình Hamilton bắt đầu từ đỉnh [C1]


In [28]:
# Tìm đường đi Euler
start = 'A2'
euler_path = find_euler_path(adjacency_list, start)

print("Đường đi Euler:")
print(' -> '.join(euler_path))

# In thống kê
print(f"Số đỉnh trong đường đi: {len(euler_path)}")
edges_count = sum(len(adjacency_list[v]) for v in adjacency_list) // 2
print(f"Tổng số cạnh trong đồ thị: {edges_count}")

Đường đi Euler:
A2 -> A1 -> A4 -> A3 -> A2
Số đỉnh trong đường đi: 5
Tổng số cạnh trong đồ thị: 36


In [29]:
# Sử dụng hàm BFS
start = 'A2'
bfs_traversal = bfs(adjacency_list, start)

# In kết quả
print(f"Thứ tự duyệt BFS :")
print(' --> '.join(bfs_traversal))

# Thống kê
print(f"\nTổng số đỉnh đã duyệt: {len(bfs_traversal)}")
print(f"Đỉnh bắt đầu: {start}")

Thứ tự duyệt BFS :
A2 --> A1 --> A3 --> A4

Tổng số đỉnh đã duyệt: 4
Đỉnh bắt đầu: A2
